<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/2020m0/2%20Advanced/342ICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ICA=Independent Component Analysis

http://scikit-learn.org/stable/auto_examples/decomposition/plot_ica_blind_source_separation.html

ICAは、複数の時系列信号の混合信号を、その統計的特徴をもとに分離する技術。例えば、3つの楽器が同時に演奏されているのを、離れた場所にある3つ(以上)のマイクで拾うと、楽器ごとの音に復元できる。

### なぜそんなことができる?

もとの信号(楽器の場合であれば演奏音)は、別個に発生しているので、互いに独立である。一番単純なケースとして、2つの音源がある場合を考えよう。1つめの音源の音を$S_1(t)$とあらわすと、マイク$j$で観測される信号は、複数の信号源からの信号の混合で、次のように書ける。(マイクと音源の相対位置は変化しないものとする)
$$O_1(t)=a_{11}S_1(t)+a_{21}S_2(t)$$
$a$は混合比である。もう一つのマイクの音源で拾う音を$O_2$とすると、連立方程式が得られる。行列の形で書くなら、
$$O=AS$$
と書ける。これより、
$$S=A^{-1}O$$
となるような行列$A$が決められれば、観測信号$O(t)$から音源信号$S(t)$を推定できる。

そこに、独立性の条件を入れる。つまり、$A$は未知なのだが、うまく係数を調節すれば、$S_1$と$S_2$を独立にできるはずである。2つの信号列の独立性は相互情報量で計量できるので、相互情報量が最小になるように$A$の係数の最適化を実行する。

### 何に使える?

信号源の数以上のセンサーがあり、信号源が信号を出しつづけていて、独立と仮定できるなら、さまざまな用途に利用できる。

* 頭に大量の脳波センサーを付けることで、脳内から発出する信号を全部分離して捉える。
* 海に音響ブイを多数配置することで、潜水艦の位置をすべて把握する。

### 実装例

下の例では、矩形波とサイン波と三角波(いずれもノイズが載っている)を合成し、3通りに混ぜあわせた混合音を作って、それをICA(独立成分分析)とPCA(主成分分析)で分離している。PCAは音源を全く分離できないが、ICAは完全な分離に成功している(ただし、原理的にもとの音量や正負は再現できない)。


In [ ]:
#Modified from:
#http://scikit-learn.org/stable/auto_examples/decomposition/plot_ica_blind_source_separation.html

print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from sklearn.decomposition import FastICA, PCA

###############################################################################
# Generate sample data
np.random.seed(0)
n_samples = 2000
time = np.linspace(0, 8, n_samples)

s1 = np.sin(2 * time)  # Signal 1 : sinusoidal signal
s2 = np.sign(np.sin(3 * time))  # Signal 2 : square signal
s3 = signal.sawtooth(2 * np.pi * time)  # Signal 3: saw tooth signal

S = np.c_[s1, s2, s3]
S += 0.2 * np.random.normal(size=S.shape)  # Add noise

S /= S.std(axis=0)  # Standardize data
# Mix data
A = np.array([[1, 1, 1], [0.5, 2, 1.0], [1.5, 1.0, 2.0]])  # Mixing matrix
X = np.dot(S, A.T)  # Generate observations

# Compute ICA
ica = FastICA(n_components=3)
S_ = ica.fit_transform(X)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

# We can `prove` that the ICA model applies by reverting the unmixing.
assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

# For comparison, compute PCA
pca = PCA(n_components=3)
H = pca.fit_transform(X)  # Reconstruct signals based on orthogonal components

###############################################################################
# Plot results

plt.figure()

models = [X, S, S_, H]
names = ['Observations (mixed signal)',
         'True Sources',
         'ICA recovered signals', 
         'PCA recovered signals']
colors = ['red', 'steelblue', 'orange']

for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(4, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()